# Feature Selection and Feature Engineering

In [1]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

## Agenda

SWBAT:

- Use correlations and other algorithms to inform feature selection;
- Address the problem of multicollinearity in regression problems;
- Create new features for use in modeling;
    - Use PolynomialFeatures to build compound features

## Model Selection

Let's imagine that I'm going to try to predict wine quality based on the other features.

Now: Which columns (predictors) should I choose? There are 12 predictors I could choose from. For each of these predictors, I could either use it or not use it in my model, which means that there are 2^12 = 4096 different models I could construct! Well, okay, one of these is the "empty model" with no predictors in it. But there are still 4095 models from which I can choose.

How can I decide which predictors to use in my model?

We'll explore a few methods here.

In [ ]:
wine = pd.read_csv('data/wine.csv')

In [ ]:
wine.head(10)

### Correlation

In [ ]:
# Use the .corr() DataFrame method to find out about the
# correlation values between all pairs of variables!

wine.corr()

In [ ]:
sns.set(rc={'figure.figsize':(8, 8)})

# Use the .heatmap method to depict the relationships visually!
sns.heatmap(wine.corr());

In [ ]:
# Let's look at the correlations with 'quality'
# (our dependent variable) in particular.

wine_corrs = wine.corr()['quality'].map(abs).sort_values(ascending=False)
wine_corrs

In [ ]:
# Let's choose 'alcohol' and 'density'.

wine_preds = wine[['alcohol', 'density']]
wine_target = wine['quality']

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(wine_preds, wine_target)

In [ ]:
lr.score(wine_preds, wine_target)

## Multicollinearity

Multicollinearity describes the correlation between distinct predictors. Why might high multicollinearity be a problem for interpreting a linear regression model?

It's problematic for statistics in an inferential mode because, if $x_1$ and $x_2$ are highly correlated with $y$ but also *with each other*, then it will be very difficult to tease apart the effects of $x_1$ on $y$ and the effects of $x_2$ on $y$. If I really want to have a good sense of the effect of $x_1$ on $y$, then I'd like to vary $x_1$ while keeping the other features constant. But if $x_1$ is highly correlated with $x_2$ then this will be a practically impossible exercise!

We will return to this topic again. For more, see [this post](https://towardsdatascience.com/https-towardsdatascience-com-multicollinearity-how-does-it-create-a-problem-72956a49058).

**A further assumption for multiple linear regression is**:

**5. The predictors are independent.**

**How can I check for this?**
- Check the model Condition Number.
- Check the correlation values.
- Compute Variance Inflation Factors ([VIFs](https://www.statsmodels.org/devel/generated/statsmodels.stats.outliers_influence.variance_inflation_factor.html)).

**What can I do if it looks like I'm violating this assumption?**

- Consider dropping offending predictors.
- We'll have much more to say about this topic in future lessons!

## Recursive Feature Elimination

The idea behind recursive feature elimination is to start with all predictive features and then build down to a small set of features slowly, by eliminating the features with the lowest coefficients.

That is:
1. Start with a model with _all_ $n$ predictors;
2. find the predictor with the smallest coefficient;
3. throw that predictor out and build a model with the remaining $n-1$ predictors;
4. set $n = n-1$ and repeat until $n-1$ has the value you want!

### Recursive Feature Elimination in Scikit-Learn

In [ ]:
lr_rfe = LinearRegression()
select = RFE(lr_rfe, n_features_to_select=3)

In [ ]:
ss = StandardScaler()
ss.fit(wine.drop('quality', axis=1))

wine_scaled = ss.transform(wine.drop('quality', axis=1))

In [ ]:
select.fit(X=wine_scaled, y=wine['quality'])

In [ ]:
select.support_

In [ ]:
wine.head()

In [ ]:
select.ranking_

These features are volatile acidity, alcohol, and red_wine.

Caution: RFE is probably not a good strategy if your initial dataset has many predictors. It will likely be easier to start with a *simple* model and then slowly increase its complexity. This is also good advice for when you're first getting your feet wet with `sklearn`!

For more on feature selection, see [this post](https://towardsdatascience.com/the-5-feature-selection-algorithms-every-data-scientist-need-to-know-3a6b566efd2).

## Feature Engineering

Domain knowledge can be helpful here! Since I don't know much about wine, I'm really just guessing.

In practice this aspect of data preparation can constitute a huge part of the data scientist's work. As we move into data modeling, much of the goal will be a matter of finding––**or creating**––features that are predictive of the targets we are trying to model. There are infinitely many ways of transforming and combining a starting set of features. Good data scientists will have a nose for which engineering operations will be likely to yield fruit and for which operations won't. And part of the game here may be getting someone else on your team who understands what the data represent better than you!

In [ ]:
wine.head(10)

### EDA

#### Chlorides

Let's look at the distribution of the chlorides feature:

In [ ]:
wine['chlorides'].hist(bins=20);

In [ ]:
wine.describe()

We'll try building a feature that records whether the level of chlorides is greater than 0.065:

In [ ]:
wine['high_chlorides'] = wine['chlorides'] > 0.065

Now we can check the correlation of this new feature with the target:

In [ ]:
wine.corr()['quality']['high_chlorides']

Not bad! We don't seem to have stumbled onto a huge connection here, but this correlation value suggests that this new feature may be helpful in a final model.

#### $\bf{SO_2}$

Next we'll take a look at distribution of the sulfur dioxide feature:

In [ ]:
wine['total sulfur dioxide'].hist(bins=25);

Let's try separating our wines into those with sulfur dioxide higher than 80 and those with less:

In [ ]:
wine['high_so2'] = wine['total sulfur dioxide'] > 80

In [ ]:
wine.corr()['quality']['high_so2']

Not great. Perhaps this is a modeling dead end.

### Products of Features

Another engineering strategy we might try is **multiplying features together**.

Earlier we found that `alcohol` and `density` had pretty high correlations with `quality`. Let's try multiplying those features together:

In [ ]:
wine['a+d'] = wine['alcohol'] * wine['density']

In [ ]:
wine.corr()['quality']['a+d']

In [ ]:
wine.corr()['quality']['alcohol']

In this case the product is only as good as one of the factors individually. But in other contexts this can be a highly effective strategy.

## PolynomialFeatures

Instead of just multiplying features at random, we might consider trying **every possible product of features**. That's what PolynomialFeatures does:

In [ ]:
pf = PolynomialFeatures(degree=3)

X = wine.drop('quality', axis=1)
y = wine['quality']

# Fitting the PolynomialFeatures object
pf.fit(X)

In [ ]:
pdf = pd.DataFrame(pf.transform(X), columns=pf.get_feature_names())
pdf

In [ ]:
pdf.shape

In [ ]:
lr = LinearRegression()

lr.fit(pdf, y)

In [ ]:
lr.score(pdf, y)

So: Is this a good idea? What are the potential dangers here?

## Exercise

Consider the following dataset:

In [ ]:
sales = pd.read_csv('data/Advertising.csv', index_col=0)

sales.head()

We'd like to try to understand sales as a function of spending on various media (TV, radio, newspaper).

In [ ]:
sales.corr()['sales']

**Try to find the best multiplicative combination of features.**

You may use `PolynomialFeatures` or just multiply by hand.

In practice, it's not easy to tell when such products of features will be so fruitful. Moreover, there is room for concern about violating regression's demand for feature independence. At the very least, we would probably not want to include a product *and the individual features themselves* in a final model, not if our goal is to understand what's really responsible for fluctuations in our target variable.